In [13]:
import numpy as np
import h5py
import sys


sys.path.append(".")
dirname = "./bgw_files/k-1x1x1/"

In [14]:
# epsinp and sigmain data.
from qtm.interfaces.bgw.epsinp import Epsinp
epsinp = Epsinp.from_epsilon_inp(filename=dirname+'epsilon.inp')

from qtm.interfaces.bgw.sigmainp import Sigmainp
sigmainp = Sigmainp.from_sigma_inp(filename=dirname+'sigma.inp')

In [15]:
from qtm.interfaces.bgw import inp
from qtm.interfaces.bgw.wfn2py import wfn2py

# Read wfndata
wfndata = wfn2py(dirname + "WFN_co.h5")
wfnqdata = wfn2py(dirname + "WFNq.h5")

# Read rho
rho = inp.read_rho(dirname + "RHO")

# Read vxc 
vxc = inp.read_vxc(dirname + "vxc.dat") 

In [16]:
from qtm.gw.core import QPoints
from qtm.gw.epsilon import Epsilon

epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)

Vcoul calculation for qpts: 100%|██████████| 1/1 [00:00<00:00, 6636.56it/s]


In [17]:
from tqdm import trange
from qtm.gw.core import reorder_2d_matrix_sorted_gvecs, sort_cryst_like_BGW


def calculate_epsilon(numq=None, writing=False):
    epsmats = []
    if numq is None:
        numq = epsilon.qpts.numq

    for i_q in trange(0, numq, desc="Epsilon> q-pt index"):
        # Create map between BGW's sorting order and QTm's sorting order
        gkspc = epsilon.l_gq[i_q]
        
        # if i_q == epsilon.qpts.index_q0:
        #     key = gkspc.g_norm2
        # else:
        #     key = gkspc.gk_norm2

        key = gkspc.gk_norm2

        indices_gspace_sorted = sort_cryst_like_BGW(
            cryst=gkspc.g_cryst, key_array=key
        )
       

        # Calculate matrix elements
        M = next(epsilon.matrix_elements(i_q=i_q))

        # Calculate polarizability matrix (faster, but not memory-efficient)
        chimat = epsilon.polarizability(M)

        # Calculate polarizability matrix (memory-efficient)
        # chimat = epsilon.polarizability_active(i_q)

        # Calculate epsilon inverse matrix
        epsinv0 = epsilon.epsilon_inverse(i_q=i_q, polarizability_matrix=chimat, store=True)
        epsinv = reorder_2d_matrix_sorted_gvecs(epsinv0, indices_gspace_sorted)

        epsilon.l_epsinv[i_q] = epsinv

        
        # Compare the results with BGW's results
        if i_q == epsilon.qpts.index_q0:
            epsref = epsilon.read_epsmat(dirname + "eps0mat.h5")[0][0, 0]
            if writing:
                epsilon.write_epsmat(
                    filename="test/epsilon/eps0mat_qtm.h5", epsinvmats=[epsinv]
                )
        else:
            epsref = np.array(epsilon.read_epsmat(dirname + "epsmat.h5")[i_q - 1][0, 0])
            epsmats.append(epsinv)

        # Calculate stddev between reference and calculated epsinv matrices
        std_eps = np.std(epsref - epsinv) / np.sqrt(np.prod(list(epsinv.shape)))


        epstol = 1e-16
        if np.abs(std_eps) > epstol:
            print(f"Standard deviation exceeded {epstol} tolerance: {std_eps}, for i_q:{i_q}")


    if writing:
        epsilon.write_epsmat(filename="test/epsilon/epsmat_qtm.h5", epsinvmats=epsmats)


epsinp.no_min_fftgrid = True
epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)

calculate_epsilon()

Epsilon> q-pt index: 100%|██████████| 1/1 [00:00<00:00, 376.78it/s]

Standard deviation exceeded 1e-16 tolerance: 1.9894890166455426e-15, for i_q:0


In [18]:
from kernel import KernelMtxEl

q0val = [0.001, 0.001, 0.001]
l_qpts = np.array(epsinp.qpts)
l_qpts[0] *= 0
qpts = QPoints.from_cryst(wfndata.kpts.recilat, None, *l_qpts)


In [19]:
kernelclass = KernelMtxEl.from_BGW(
    wfndata=wfndata,
    epsinp=epsinp,
    sigmainp=sigmainp,
    l_epsmats=epsilon.l_epsinv,
    q0val=q0val,
    parallel=False,
    num_bands_val=1,
    num_bands_con=1,
)

In [20]:
data = kernelclass.calc_kernel_mtxel()

In [21]:
exc = data["exc"]
print(f"The exc kernel is {exc}")

head = data["head"]
print(f"The head kernel is {head}")

wings = data["wings"]
print(f"The wings kernel is {wings}")

body = data["body"]
print(f"The body kernel is {body}")

The exc kernel is [[[[[[0.39326425+2.77555756e-17j]]]]]]
The head kernel is [[[[[[1.-1.89016773e-19j]]]]]]
The wings kernel is [[[[[[0.+0.j]]]]]]
The body kernel is [[[[[[-0.020828-9.12308222e-09j]]]]]]


Calculate $W_{G, G'}(q, 0)$ 

In [22]:
norm_array = kernelclass.l_gq[0].g_norm2
vq0g = np.where(norm_array == 0, 0, 1 / np.where(norm_array == 0, 1, norm_array))
sort_order_0 = sort_cryst_like_BGW(
    kernelclass.l_gq[0].g_cryst, norm_array
)
vq0g = vq0g[sort_order_0]

for i, value in enumerate(vq0g):
    print(f"Row {i+1}: {value}")

Row 1: 0.0
Row 2: 0.8784546621790682
Row 3: 0.8784546621790684
Row 4: 0.8784546621790684
Row 5: 0.8784546621790684
Row 6: 0.8784546621790684
Row 7: 0.8784546621790684
Row 8: 0.8784546621790684
Row 9: 0.8784546621790682
Row 10: 0.6588409966343013
Row 11: 0.6588409966343013
Row 12: 0.6588409966343013
Row 13: 0.6588409966343013
Row 14: 0.6588409966343013
Row 15: 0.6588409966343013
Row 16: 0.3294204983171506
Row 17: 0.3294204983171506
Row 18: 0.3294204983171506
Row 19: 0.32942049831715065
Row 20: 0.32942049831715065
Row 21: 0.32942049831715065
Row 22: 0.32942049831715065
Row 23: 0.32942049831715065
Row 24: 0.32942049831715065
Row 25: 0.3294204983171506
Row 26: 0.3294204983171506
Row 27: 0.3294204983171506


In [23]:
i_q = 0
epsinv = kernelclass.l_epsmats[i_q]
epsinv = np.conjugate(epsinv)
    
sort_order = sort_cryst_like_BGW(
    kernelclass.l_gq_epsinv[i_q].gk_cryst, kernelclass.l_gq_epsinv[i_q].gk_norm2
)
            
# As given in the sigma code, I need to use another sorting.
sort_order_QTM = np.argsort(sort_order)

# But my matrices, vcoul, etc. are sorted according to BGW.
sort_BGW = sort_cryst_like_BGW(
    kernelclass.l_gq_epsinv[i_q].g_cryst, kernelclass.l_gq_epsinv[i_q].g_norm2
)

sort_order_BGW = sort_order_QTM[sort_BGW]

epsiloninv = (
    reorder_2d_matrix_sorted_gvecs(epsinv, sort_order_BGW)
)

# Print epsinv in rows and columns
# Exclude the first row and column
epsiloninv_excluded = epsiloninv[1:, 1:]

# Print the modified epsinv in rows and columns
for row_idx, row in enumerate(epsiloninv_excluded):
    for col_idx, value in enumerate(row):
        print(f"Row {row_idx + 2}, Column {col_idx + 2}: {value}")

Row 2, Column 2: (0.8232266880027871+6.776263578034403e-21j)
Row 2, Column 3: (-0.01066540937404379-2.1876786619713103e-06j)
Row 2, Column 4: (-0.01067328136776466-3.1219933458714887e-06j)
Row 2, Column 5: (-0.010678995673843995+1.810786966767277e-06j)
Row 2, Column 6: (0.010870698485405965-2.4598136379729925e-07j)
Row 2, Column 7: (0.010866699331034228+5.5997582124294636e-08j)
Row 2, Column 8: (0.010857408093462877-1.7971828645745793e-06j)
Row 2, Column 9: (0.1761034191796887-1.3605386655277362e-06j)
Row 2, Column 10: (-0.040945972089021246+1.1106146816448985e-06j)
Row 2, Column 11: (-0.04094610269179326-2.5548732637087e-06j)
Row 2, Column 12: (-0.040946403020512494-1.3102758526604763e-06j)
Row 2, Column 13: (0.040993512375649105-3.202891993604327e-06j)
Row 2, Column 14: (0.040993130578149906+5.483721606302487e-07j)
Row 2, Column 15: (0.0409930953189592+2.112491441857268e-07j)
Row 3, Column 2: (-0.010679658403686109+2.190601409426097e-06j)
Row 3, Column 3: (0.7880264851938111+1.058791

In [24]:
wcoul = np.zeros_like(epsiloninv)

wcoul[0, 0] = 1
wcoul[0, 1:] = 0
wcoul[1:, 0] = 0

for i in range(1, len(wcoul)):
    for j in range(1, len(wcoul)):
        wcoul[i, j] = epsiloninv[i, j] * vq0g[j]

# Print wcoul in rows and columns
wcoul_excluded = wcoul[1:, 1:]
# Print the modified wcoul in rows and columns
for row_idx, row in enumerate(wcoul_excluded):
    for col_idx, value in enumerate(row):
        print(f"Row {row_idx + 2}, Column {col_idx + 2}: {value}")

Row 2, Column 2: (0.7231673221062815+5.952640332278535e-21j)
Row 2, Column 3: (-0.009369078588677106-1.9217765199583636e-06j)
Row 2, Column 4: (-0.00937599377826185-2.7425296099728382e-06j)
Row 2, Column 5: (-0.00938101353707836+1.5906942531698084e-06j)
Row 2, Column 6: (0.009549415765647808-2.1608347583690303e-07j)
Row 2, Column 7: (0.009545902689845181+4.919133708784189e-08j)
Row 2, Column 8: (0.009537740758883214-1.5787436661738726e-06j)
Row 2, Column 9: (0.15469886960407228-1.1951715338077276e-06j)
Row 2, Column 10: (-0.02697688505929104+7.317184837316122e-07j)
Row 2, Column 11: (-0.026976971105751518-1.68325524733617e-06j)
Row 2, Column 12: (-0.026977168974624217-8.632634486326872e-07j)
Row 2, Column 13: (0.02700820654911322-2.110196553178299e-06j)
Row 2, Column 14: (0.027007955005268335+3.612900608361382e-07j)
Row 2, Column 15: (0.02700793177506799+1.3917959669346746e-07j)
Row 3, Column 2: (-0.009381595715197928+1.924344021086393e-06j)
Row 3, Column 3: (0.692245539839088+9.301000